## Method 1: Python Sort 

In [8]:
import pandas as pd

# This function calculates how much each debtor owes to the payer
def calculate_debts_python_sorted(expenses):
    balances = {}

    for expense in expenses:
        # The amount owed per debtor is the total amount divided by the number of debtors
        amount_per_debtor = expense.amount / len(expense.debtors)
        for debtor in expense.debtors:
            if debtor != expense.payer:  # Payer should not owe money to themselves
                # Update the debtor's balance
                balances[debtor] = balances.get(debtor, 0) - amount_per_debtor
                # Update the payer's balance
                balances[expense.payer] = balances.get(expense.payer, 0) + amount_per_debtor

    balances_copy = balances.copy()
    sorted_balances = sorted(balances_copy.items(), key=lambda x: x[1])
    debtors = 0
    credictors = len(sorted_balances) - 1
    transactions = []

    while any(x != 0 for x in [value for key, value in sorted_balances]) and debtors < credictors:
        debtor, debtor_balance = sorted_balances[debtors]
        creditor, creditor_balance = sorted_balances[credictors]

        if debtor_balance == 0:
            debtors += 1
            continue
        if creditor_balance == 0:
            creditors -= 1
            continue

        transfer_amount = min(-debtor_balance, creditor_balance)
        debtor_balance += transfer_amount
        creditor_balance -= transfer_amount

        transactions.append((debtor, creditor, transfer_amount))

        sorted_balances[debtors] = (debtor, debtor_balance)
        sorted_balances[credictors] = (creditor, creditor_balance)

        if debtor_balance == 0:
            debtors += 1
        if creditor_balance == 0:
            credictors -= 1

    return transactions


# Example usage with hardcoded data

class Expense:
    def __init__(self, payer, debtors, amount):
        self.payer = payer
        self.debtors = debtors
        self.amount = amount

expenses = [
    Expense('elena', ['elena', 'minho', 'lino'], 1000.0),
    Expense('minho', ['elena', 'minho'], 500.0),
    Expense('minho', ['minho'], 500.0),
    Expense('elena', ['elena', 'minho'], 100.0),
    Expense('lino', ['elena', 'minho', 'lino'], 10.0)
]


debt_table = calculate_debts_python_sorted(expenses)

# Convert the list of transactions to a pandas DataFrame for tabular display
df = pd.DataFrame(debt_table)

# Print the DataFrame as a table
print(df.to_string(index=False))


# Test
assert df.shape[1] >= df[0].nunique() #the number of transactions is at least the number of debtors
assert (df[2] < 0).sum() == 0 #transactions values cannot be negatve

    0     1          2
 lino elena 326.666667
minho elena 136.666667


## Method 2: Merge Sort

In [9]:

def calculate_debts_merge_sort(expenses):
    balances = {}

    for expense in expenses:
        amount_per_debtor = expense.amount / len(expense.debtors)
        for debtor in expense.debtors:
            if debtor != expense.payer:  # Skip if the debtor is also the payer
                balances[debtor] = balances.get(debtor, 0) - amount_per_debtor
                balances[expense.payer] = balances.get(expense.payer, 0) + amount_per_debtor

    balances_copy = balances.copy()
    sorted_balances = list(merge_sort_dict_by_value(balances_copy).items())
    debtors = 0
    credictors = len(sorted_balances) - 1
    transactions = []

    while any(x != 0 for x in [value for key, value in sorted_balances]) and debtors < credictors:
        debtor, debtor_balance = sorted_balances[debtors]
        creditor, creditor_balance = sorted_balances[credictors]

        if debtor_balance == 0:
            debtors += 1
            continue
        if creditor_balance == 0:
            creditors -= 1
            continue

        transfer_amount = min(-debtor_balance, creditor_balance)
        debtor_balance += transfer_amount
        creditor_balance -= transfer_amount

        transactions.append((debtor, creditor, transfer_amount))

        sorted_balances[debtors] = (debtor, debtor_balance)
        sorted_balances[credictors] = (creditor, creditor_balance)

        if debtor_balance == 0:
            debtors += 1
        if creditor_balance == 0:
            credictors -= 1

    return transactions

def merge_sort(arr, compare_func=None):
    arr_temp = list(arr)
    n = len(arr_temp)    

    if n > 1: 
        mid = n // 2
        arr_temp_left = arr_temp[:mid] 
        arr_temp_right = arr_temp[mid:]
  
        arr_temp_left = merge_sort(arr_temp_left, compare_func)
        arr_temp_right = merge_sort(arr_temp_right, compare_func)
          
        i = j = k = 0
        n_left, n_right = len(arr_temp_left), len(arr_temp_right)
          
        while i < n_left and j < n_right: 
            if compare_func(arr_temp_left[i], arr_temp_right[j]):
                arr_temp[k] = arr_temp_left[i] 
                i += 1
            else: 
                arr_temp[k] = arr_temp_right[j] 
                j += 1
            k += 1
          
        while i < n_left: 
            arr_temp[k] = arr_temp_left[i] 
            i += 1
            k += 1
 
        while j < n_right: 
            arr_temp[k] = arr_temp_right[j] 
            j += 1
            k += 1
            
    return arr_temp


def merge_sort_dict_by_value(dictionary):
    def compare_dict_items(item1,item2):
        return item1[1] < item2[1]

    items = list(dictionary.items())
    sorted_items = merge_sort(items, compare_func=compare_dict_items)
    return dict(sorted_items)

class Expense:
    def __init__(self, payer, debtors, amount):
        self.payer = payer
        self.debtors = debtors
        self.amount = amount

expenses = [
    Expense('elena', ['elena', 'minho', 'lino'], 1000.0),
    Expense('minho', ['elena', 'minho'], 500.0),
    Expense('minho', ['minho'], 500.0),
    Expense('elena', ['elena', 'minho'], 100.0),
    Expense('lino', ['elena', 'minho', 'lino'], 10.0)
]

debt_table = calculate_debts_merge_sort(expenses)

# Convert the list of transactions to a pandas DataFrame for tabular display
df = pd.DataFrame(debt_table)

print(df.to_string(index=False))


# Test
assert df.shape[1] >= df[0].nunique() #the number of transactions is at least the number of debtors
assert (df[2] < 0).sum() == 0 #transactions values cannot be negatve

    0     1          2
 lino elena 326.666667
minho elena 136.666667
